# Preprocess and Feature Extraction - Flavia dataset

Extracted features are saved in file named "Flavia_features.csv"

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import mahotas as mt
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
ds_path = "Leaves"
img_files = os.listdir(ds_path)

In [3]:
def create_dataset():
    names = ['area','perimeter','physiological_length','physiological_width','aspect_ratio','rectangularity','circularity', \
             'mean_r','mean_g','mean_b','stddev_r','stddev_g','stddev_b', \
             'contrast','correlation','inverse_difference_moments','entropy'
            ]
    df = pd.DataFrame([], columns=names)
    for file in img_files:
        imgpath = ds_path + "/" + file
        main_img = cv2.imread(imgpath)
        
        #Preprocessing
        img = cv2.cvtColor(main_img, cv2.COLOR_BGR2RGB)
        gs = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        blur = cv2.GaussianBlur(gs, (25,25),0)
        ret_otsu,im_bw_otsu = cv2.threshold(blur,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        kernel = np.ones((50,50),np.uint8)
        closing = cv2.morphologyEx(im_bw_otsu, cv2.MORPH_CLOSE, kernel)
        
        #Shape features
        image, contours, _ = cv2.findContours(closing,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        cnt = contours[0]
        M = cv2.moments(cnt)
        area = cv2.contourArea(cnt)
        perimeter = cv2.arcLength(cnt,True)
        x,y,w,h = cv2.boundingRect(cnt)
        aspect_ratio = float(w)/h
        rectangularity = w*h/area
        circularity = ((perimeter)**2)/area
        
        #Color features
        red_channel = img[:,:,0]
        green_channel = img[:,:,1]
        blue_channel = img[:,:,2]
        blue_channel[blue_channel == 255] = 0
        green_channel[green_channel == 255] = 0
        red_channel[red_channel == 255] = 0
        
        red_mean = np.mean(red_channel)
        green_mean = np.mean(green_channel)
        blue_mean = np.mean(blue_channel)
        
        red_std = np.std(red_channel)
        green_std = np.std(green_channel)
        blue_std = np.std(blue_channel)
        
        #Texture features
        textures = mt.features.haralick(gs)
        ht_mean = textures.mean(axis=0)
        contrast = ht_mean[1]
        correlation = ht_mean[2]
        inverse_diff_moments = ht_mean[4]
        entropy = ht_mean[8]
        
        vector = [area,perimeter,w,h,aspect_ratio,rectangularity,circularity,\
                  red_mean,green_mean,blue_mean,red_std,green_std,blue_std,\
                  contrast,correlation,inverse_diff_moments,entropy
                 ]
        
        df_temp = pd.DataFrame([vector],columns=names)
        df = df.append(df_temp)
        print(file)
    return df

In [ ]:
dataset = create_dataset()

2191.jpg
3245.jpg
3382.jpg
3552.jpg
3359.jpg
1353.jpg
3478.jpg
1011.jpg
1457.jpg
1519.jpg
2339.jpg
2403.jpg
2664.jpg
2544.jpg
1067.jpg
3170.jpg
2374.jpg
3010.jpg
1339.jpg
1044.jpg
3063.jpg
3152.jpg
1600.jpg
3437.jpg
3098.jpg
2026.jpg
2515.jpg
1483.jpg
2648.jpg
3069.jpg
1195.jpg
3489.jpg
1170.jpg
2093.jpg
1259.jpg
3530.jpg
1432.jpg
2602.jpg
2147.jpg
3449.jpg
1280.jpg
1597.jpg
1400.jpg
2344.jpg
2309.jpg
1292.jpg
1258.jpg
2103.jpg
3576.jpg
3433.jpg
3191.jpg
3299.jpg
1048.jpg
3574.jpg
2065.jpg
1108.jpg
3041.jpg
1531.jpg
2033.jpg
1546.jpg
3554.jpg
3427.jpg
2393.jpg
3204.jpg
2286.jpg
2426.jpg
2308.jpg
2082.jpg
3240.jpg
3463.jpg
2140.jpg
2669.jpg
3386.jpg
3607.jpg
1112.jpg
2514.jpg
1156.jpg
3555.jpg
2288.jpg
3283.jpg
1121.jpg
1257.jpg
2229.jpg
2022.jpg
2099.jpg
1311.jpg
1263.jpg
3496.jpg
1496.jpg
1077.jpg
2177.jpg
1075.jpg
3353.jpg
2370.jpg
3566.jpg
2538.jpg
3144.jpg
2480.jpg
1136.jpg
2313.jpg
3344.jpg
2289.jpg
1387.jpg
1550.jpg
2446.jpg
2504.jpg
1355.jpg
3286.jpg
3149.jpg
1160.jpg
3171.jpg
3

In [ ]:
dataset.shape

In [ ]:
type(dataset)

In [ ]:
dataset.to_csv("Flavia_features.csv")